In [1]:
import datetime
import inspect
import json
from typing import List, Dict, Literal, Callable, Any, TypedDict
from typing import get_type_hints

from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import (
    ChatCompletionMessageToolCall,
)

In [2]:
load_dotenv()
client = OpenAI()

### Memory Layer

- Detailed explanation of the memory layer see previous notebook [04-react-agent-from-scratch.ipynb](./04-react-agent-from-scratch.ipynb).

In [6]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []

    def add_message(
        self,
        role: Literal[
            "user", "system", "assistant", "tool"
        ],  # Added 'tool' as a new role type
        content: str,
        tool_calls: List = None,  # New parameter to store tool call information
        tool_call_id=None,
    ) -> None:  # New parameter to track specific tool call IDs

        # For regular messages (user/system/assistant), include tool_calls dictionary
        message = {"role": role, "content": content}

        # Only add tool_calls if they exist and role is assistant
        if tool_calls and role == "assistant":
            message["tool_calls"] = tool_calls

        # Add tool_call_id if it exists and role is tool
        if tool_call_id and role == "tool":
            message["tool_call_id"] = tool_call_id

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    def reset(self) -> None:
        self._messages = []

### Helper Functions for Tool

- Detailed explanation, see previous notebook [04-react-agent-from-scratch.ipynb](./04-react-agent-from-scratch.ipynb).

In [ ]:
def get_func_name(func: Callable) -> str:
    return func.__name__


def get_func_desc(func: Callable) -> str:
    if func.__doc__:
        return func.__doc__.strip()
    else:
        return get_func_name(func)


def infer_json_type(arg_type: Any) -> str:
    """
    Infers the JSON schema type from a given Python type.

    Parameters:
    - arg_type: The Python type to be mapped to a JSON schema type.

    Returns:
    - str: The corresponding JSON schema type as a string.
    """
    type_mapping = {
        bool: "boolean",
        int: "integer",
        float: "number",
        str: "string",
        list: "array",
        dict: "object",
        type(None): "null",
        datetime.date: "string",
        datetime.datetime: "string",
    }

    # Check if arg_type is directly in the mapping
    if arg_type in type_mapping:
        return type_mapping[arg_type]

    # If arg_type is a subclass of a mapped type, return the mapped type
    for base_type in type_mapping:
        if isinstance(arg_type, base_type):
            return type_mapping[base_type]

    # Default to string if type is unknown
    return "string"


class FuncArgument(TypedDict):
    name: str
    type: str
    required: bool


def get_func_args(func: Callable) -> List[FuncArgument]:
    args_type_mapping = get_type_hints(func)
    signature = inspect.signature(func)

    def is_required(param: inspect.Parameter) -> bool:
        return param.default == inspect.Parameter.empty

    arguments = []
    for arg_name, arg_type in args_type_mapping.items():
        param = signature.parameters.get(arg_name)
        if param:
            arguments.append(
                {
                    "name": arg_name,
                    "type": infer_json_type(arg_type),
                    "required": is_required(param),
                }
            )

    return arguments

### Tool Layer

- Detailed explanation, see previous notebook [04-react-agent-from-scratch.ipynb](./04-react-agent-from-scratch.ipynb).

In [5]:
class Tool:
    def __init__(self, func: Callable):
        self.func = func
        self.name: str = get_func_name(func)
        self.description: str = get_func_desc(func)
        self.arguments: List[FuncArgument] = get_func_args(func)

    def to_dict(self):
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parallel_tool_calls": False,
                "parameters": {
                    "type": "object",
                    "properties": {
                        argument["name"]: {
                            "type": argument["type"],
                        }
                        for argument in self.arguments
                    },
                    "required": [
                        argument["name"]
                        for argument in self.arguments
                        if argument["required"]
                    ],
                    "additionalProperties": False,
                },
                "strict": True,
            },
        }

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

### ReAct Prompt with Another Agent

In previous notebook [04-react-agent-from-scratch.ipynb](./04-react-agent-from-scratch.ipynb), we define a template to generate the ReAct prompt by combining the tool calling, like this:

```python
react_prompt = (
    "You're an AI Agent, your role is {ROLE}, " 
    "and you need to {INSTRUCTIONS} "
    "You can answer multistep questions by sequentially calling functions. "
    "You follow a pattern of of Thought and Action. "
    "Create a plan of execution: "
    "- Use Thought to describe your thoughts about the question you have been asked. "
    "- Use Action to specify one of the tools available to you. "
    "When you think it's over call the termination tool. "
    "Never try to respond directly if the question needs a tool."
    "The actions you have are the Tools: "
    "{TOOLS}"
)
```

Now, we will use the same template, but we will add a new tool to call another agent. This tool will be called `call_peer_agent` and it will be used to call another agent to perform a task.
```python
...
"The call_agents tool is to call one of the following peer agents: "
f"{PEER_AGENT} ",
```

In [7]:
def call_peer_agent(agent_name: str, message: str) -> Dict[str, str]:
    """
    Based on the task at hand and the available agents, call one to perform it.
    Tell the agent with a message the exact task it needs to perform just like if you were the user.
    """
    return {"agent_name": agent_name, "message": message}

### ReAct Loop: Reasoning and Action

We keep the same ReAct loop as before, which is:

A flow of "Think → Act → Observe → Think" that mimics human problem-solving patterns, allowing the AI to break down complex tasks into manageable steps while maintaining a record of its reasoning process.

- Detailed explanation, see previous notebook [04-react-agent-from-scratch.ipynb](./04-react-agent-from-scratch.ipynb).

But we add a new tool to call another agent. This tool will be called `call_peer_agent` and it will be used to call another agent to perform a task.


In [26]:
TERMINATION_MESSAGE = "StopReactLoopException"

class StopReactLoopException(Exception):
    """
    Terminates ReAct loop
    """

def termination() -> str:
    """Terminate the ReAct loop. If the agent thinks there's no further actions to take"""
    return TERMINATION_MESSAGE

In [35]:
class Agent:
    """A tool-calling AI Agent"""

    def __init__(
        self,
        name: str,  # This is the id of your agent, and should be unique
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
        funcs: List[Callable] = [],
        peer_agents: List["Agent"] = None,
    ):
        self.name = name if name else self._default_agent_name()
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.client = OpenAI()

        # Tools
        tools = [Tool(func) for func in funcs] + [Tool(termination)]

        # Adding peer agents to the tools and the tool map
        # This is to allow the agent to call another agent to perform a task
        self.peer_agents = peer_agents
        if peer_agents:
            tools.append(Tool(call_peer_agent))
            self.peer_agents = [
                {
                    "name": agent.name,
                    "role": agent.role,
                    "instructions": agent.instructions,
                }
                for agent in peer_agents
            ]
            self.peer_agents_map = {agent.name: agent for agent in peer_agents}

        self.tool_map = {tool.name: tool for tool in tools}
        self.tools = [tool.to_dict() for tool in tools] if tools else None
        self.termination_message = TERMINATION_MESSAGE

        # Memory
        self.memory = Memory()
        self.memory.add_message(
            role="system",
            content=f"You're an AI Agent, your role is {self.role}, "
            f"and you need to {self.instructions} "
            "You can answer multistep questions by sequentially calling functions. "
            "You follow a pattern of of Thought and Action. "
            "Create a plan of execution: "
            "- Use Thought to describe your thoughts about the question you have been asked. "
            "- Use Action to specify one of the tools available to you. "
            "When you think it's over call the termination tool. "
            "Never try to respond directly if the question needs a tool."
            "The actions you have are the Tools: "
            "```\n"
            f"{self.tools} "
            "```\n"
            "The call_agents tool is to call one of the following peer agents: "
            f"{self.peer_agents} ",
        )

    def invoke(self, user_message: str, max_iter: int = 3) -> str:
        self.memory.add_message(
            role="user",
            content=user_message,
        )
        try:
            self._react_loop(max_iter)
        except StopReactLoopException as e:
            print(f"Termninated loop with message: '{e!s}'")
            self._reason()

        return self.memory.last_message()

    def _react_loop(self, max_iter: int):
        for i in range(max_iter):
            self._reason()

            ai_message = self._get_completion(
                messages=self.memory.get_messages(),
                tools=self.tools,
            )
            tool_calls = ai_message.tool_calls

            self.memory.add_message(
                role="assistant",
                content=ai_message.content,
                tool_calls=tool_calls,
            )

            if tool_calls:
                self._call_tools(tool_calls)

    def _reason(self):
        # No tools
        ai_message = self._get_completion(
            messages=self.memory.get_messages(),
        )
        tool_calls = ai_message.tool_calls

        self.memory.add_message(
            role="assistant",
            content=ai_message.content,
            tool_calls=tool_calls,
        )

    def _call_tools(self, tool_calls: List[ChatCompletionMessageToolCall]):
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            callable_tool = self.tool_map[function_name]
            result = callable_tool(**args)

            if function_name == "call_peer_agent":
                print(result)
                agent_name = result["agent_name"]
                message = result["message"]
                peer_agent = self.peer_agents_map[agent_name]
                result = peer_agent.invoke(message)

            self.memory.add_message(
                role="tool", content=str(result), tool_call_id=tool_call.id
            )
            if result == TERMINATION_MESSAGE:
                raise StopReactLoopException

    def _get_completion(
        self, messages: List[Dict], tools: List = None
    ) -> ChatCompletionMessage:
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=messages,
            tools=tools,
        )

        return response.choices[0].message

    def _default_agent_name(self):
        for var_name, obj in globals().items():
            if obj is self:
                return var_name
        return None

In [36]:
def power(base: float, exponent: float):
    """Exponentatiation: base to the power of exponent"""

    return base**exponent

In [37]:
def sum(number_1: float, number_2: float):
    """Sum / Addition: Add two numbers"""

    return number_1 + number_2

### Create Peer Agent for Exponentiation

In [38]:
exponentiation_agent = Agent(
    name="exponentiation_agent",
    role="Do the exponentation of a base to the power of an exponent",
    instructions="Help your peers with exponentiation problems",
    funcs=[power],
)

### Create Peer Agent for Summing

In [39]:
summing_agent = Agent(
    name="summing_agent",
    role="Sum two numbers",
    instructions="Help your peers with addition problems",
    funcs=[sum],
)

### Create Agent with Peers

In [40]:
agent = Agent("assistant", peer_agents=[summing_agent, exponentiation_agent])

In [41]:
agent.invoke("What's 2 to the power of 3? Then add 10 to the result")

{'agent_name': 'exponentiation_agent', 'message': 'Calculate 2 to the power of 3.'}
Termninated loop with message: ''
{'agent_name': 'summing_agent', 'message': 'Add 8 and 10.'}
Termninated loop with message: ''
Termninated loop with message: ''


{'role': 'assistant',
 'content': 'The final result of \\(2\\) to the power of \\(3\\) plus \\(10\\) is \\(18\\). If you have any more questions or need further assistance, feel free to ask!'}

In [42]:
agent.memory.get_messages()

[{'role': 'system',
  'content': 'You\'re an AI Agent, your role is Personal Assistant, and you need to Help users with any question You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Create a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. When you think it\'s over call the termination tool. Never try to respond directly if the question needs a tool.The actions you have are the Tools: ```\n[{\'type\': \'function\', \'function\': {\'name\': \'termination\', \'description\': "Terminate the ReAct loop. If the agent thinks there\'s no further actions to take", \'parallel_tool_calls\': False, \'parameters\': {\'type\': \'object\', \'properties\': {}, \'required\': [], \'additionalProperties\': False}, \'strict\': True}}, {\'type\': \'function\', \'function\': {\'name\': \'call_peer_agent\', \'description\': \'Based o

In [43]:
agent.peer_agents_map["exponentiation_agent"].memory.get_messages()

[{'role': 'system',
  'content': 'You\'re an AI Agent, your role is Do the exponentation of a base to the power of an exponent, and you need to Help your peers with exponentiation problems You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Create a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. When you think it\'s over call the termination tool. Never try to respond directly if the question needs a tool.The actions you have are the Tools: ```\n[{\'type\': \'function\', \'function\': {\'name\': \'power\', \'description\': \'Exponentatiation: base to the power of exponent\', \'parallel_tool_calls\': False, \'parameters\': {\'type\': \'object\', \'properties\': {\'base\': {\'type\': \'number\'}, \'exponent\': {\'type\': \'number\'}}, \'required\': [\'base\', \'exponent\'], \'additionalProperties\': False}, \'stric

In [44]:
agent.peer_agents_map["summing_agent"].memory.get_messages()

[{'role': 'system',
  'content': 'You\'re an AI Agent, your role is Sum two numbers, and you need to Help your peers with addition problems You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Create a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. When you think it\'s over call the termination tool. Never try to respond directly if the question needs a tool.The actions you have are the Tools: ```\n[{\'type\': \'function\', \'function\': {\'name\': \'sum\', \'description\': \'Sum / Addition: Add two numbers\', \'parallel_tool_calls\': False, \'parameters\': {\'type\': \'object\', \'properties\': {\'number_1\': {\'type\': \'number\'}, \'number_2\': {\'type\': \'number\'}}, \'required\': [\'number_1\', \'number_2\'], \'additionalProperties\': False}, \'strict\': True}}, {\'type\': \'function\', \'function\': {\'name